# Accuracy=88.6 at Epoch=40 (18+22=40) 
# Accuracy=88.8 at Epoch=50 (18+32=50)

## 1. Train model on 16x16 Image for 18 epochs (called Pre-trained model)
## 2. Load weight from Pre-trained Model
## 3. Train model on 32x32 Image for 22 epochs(88.6%)/for 32 epochs(88.8%)  


In [0]:
# https://keras.io/
!pip install -q keras
!pip install -q scipy 

import scipy
import numpy as np
import matplotlib.pyplot as plt
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.preprocessing.image import ImageDataGenerator

import h5py
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 64
num_classes = 10
epochs = 50
l = 16
num_filter = 32
compression = 0.5
dropout_rate = 0.1


In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

## Image Rescaling for 16x16


In [0]:
#Rescaling
x_train_rescale = np.array([scipy.misc.imresize(image, (16, 16),mode='RGB') for image in x_train])
print (x_train_rescale.shape)    # (50000, 16, 16, 3)
print (x_train.shape)            # (50000, 32, 32, 3)

x_test_rescale = np.array([scipy.misc.imresize(image,(16,16),mode='RGB') for image in x_test])
print (x_test_rescale.shape)      # (10000, 16, 16, 3)
print (x_test.shape)              # (10000, 32, 32, 3)

img_height, img_width, channel = x_train_rescale.shape[1],x_train_rescale.shape[2],x_train_rescale.shape[3] #Used while training Rescaling image/Pre-train model
#img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]      # USed while training original image

#plt.imshow(x_train[0])
#plt.show()
#plt.imshow(x_train_rescale[0])
#plt.show()

/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:482: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if issubdtype(ts, int):
/usr/local/lib/python3.6/dist-packages/scipy/misc/pilutil.py:485: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  elif issubdtype(type(size), float):


(50000, 16, 16, 3)
(50000, 32, 32, 3)
(10000, 16, 16, 3)
(10000, 32, 32, 3)


## Densnet Block/Transition/Output Layer


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization(name='Batch'+str(block)+str(_))(temp)
        relu = Activation('relu',name='Activation'+str(block)+str(_))(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same',name='Conv'+str(block)+str(_))(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate,name='Dropout'+str(block)+str(_)) (Conv2D_3_3)
        concat = Concatenate(axis=-1,name='Concat'+str(block)+str(_))([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization(name='Batch'+'_transition'+str(block))(input)
    relu = Activation('relu',name='Activation'+'_transition'+str(block))(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same',name='Conv'+'_transition'+str(block))(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate,name='Dropout'+'_transition'+str(block))(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2),name='AvgPooling'+'_transition'+str(block))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization(name='Batch_output_layer')(input)
    relu = Activation('relu',name='Activation_output_layer')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2),name='AvgPooling_output_layer')(relu)
    flat = Flatten(name='Flattern_output_layer')(AvgPooling)
    output = Dense(num_classes, activation='softmax',name='Dense_output_layer')(flat)
    
    return output

In [0]:
def my_output_layer(input):
    global compression
    BatchNorm = BatchNormalization(name='Batch_output_layer')(input)
    relu = Activation('relu',name='Activation_output_layer')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2),name='AvgPooling_output_layer')(relu)
    flat = Flatten(name='Flattern_output_layer')(AvgPooling)
    output = Dense(num_classes, activation='softmax',name='my_Dense_output_layer')(flat)  #Name of Dense layer is different than output_layer's Dense layer
    
    return output

##  Data Augmentation Config

In [0]:
imgen = ImageDataGenerator(
    width_shift_range=0.125,
    height_shift_range=0.125, 
    horizontal_flip=True,
    fill_mode='constant'
)



## Desnet Model 

In [0]:
dropout_rate = 0.1
input = Input(shape=(img_height, img_width, channel,),name='input_layer')
input_norm = BatchNormalization(axis=-1,name='Channelwise_norm') (input)
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same',name='Conv2D_1st')(input_norm)

block=1
First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

block=2
Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

block=3
Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

block=4
Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [0]:
model = Model(inputs=[input], outputs=[output])

# Step1:-Train model on 16x16 image (Called Pre-trained model)

---



In [0]:
weight_checksaver = keras.callbacks.ModelCheckpoint('cifar10_weights.{epoch:02d}-{val_loss:.3f}.hdf5',
                                                  monitor='val_loss', verbose=1, save_weights_only=True, period=5, mode = 'auto')

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])


In [0]:
#Total 18 epochs with Increasing Batch size
#No. of epochs=  epcohs - initial_epochs

#Batch  epcohs  initial_epochs  No.ofepoch  
# 16      1          0             1        
# 32      3          1             2      
# 48      8          3             5
# 10      18         10            10
#---------------------------------------------------
#Total Epochs                 =    18

epochs= 1
batch_size = 16 
history1=model.fit_generator(imgen.flow(x_train_rescale,y_train,batch_size=batch_size),
                         steps_per_epoch=len(x_train_rescale)/batch_size,
                         epochs=epochs,
                         initial_epoch=0,
                         verbose=1,
                         callbacks=[weight_checksaver],
                         validation_data=(x_test_rescale,y_test)
                        )
epochs= 3  # No. of epoch=2, as initial_epoch=1 (3-2=1)
batch_size = 32
history2=model.fit_generator(imgen.flow(x_train_rescale,y_train,batch_size=batch_size),
                         steps_per_epoch=len(x_train_rescale)/batch_size,
                         epochs=epochs,
                         initial_epoch=1,
                         verbose=1,
                         callbacks=[weight_checksaver],
                         validation_data=(x_test_rescale,y_test)
                        )
epochs= 8 # No. of Epoch =5,  as initial_epoch=3 (8-3=5)
batch_size = 48
history3=model.fit_generator(imgen.flow(x_train_rescale,y_train,batch_size=batch_size),
                         steps_per_epoch=len(x_train_rescale)/batch_size,
                         epochs=epochs, 
                         initial_epoch=3,
                         verbose=1,
                         callbacks=[weight_checksaver],
                         validation_data=(x_test_rescale,y_test)
                        )
epochs= 18 # No. of Epoch =10,  as initial_epoch=8 (18-10=8)
batch_size = 64
history4=model.fit_generator(imgen.flow(x_train_rescale,y_train,batch_size=batch_size),
                        steps_per_epoch=len(x_train_rescale)/batch_size,
                        epochs=epochs, 
                        initial_epoch=8,                             
                        verbose=1,
                        callbacks=[weight_checksaver],
                        validation_data=(x_test_rescale,y_test)
                       )

Epoch 1/1
3124/3125 [============================>.] - ETA: 0s - loss: 1.7485 - acc: 0.3521

3125/3125 [==============================] - 1139s 364ms/step - loss: 1.7485 - acc: 0.3521 - val_loss: 1.5617 - val_acc: 0.4284
Epoch 2/3
 328/1562 [=====>........................] - ETA: 7:48 - loss: 1.4596 - acc: 0.4657

1562/1562 [============================>.] - ETA: 0s - loss: 1.4052 - acc: 0.4890

1563/1562 [==============================] - 608s 389ms/step - loss: 1.4052 - acc: 0.4890 - val_loss: 1.2462 - val_acc: 0.5476
Epoch 3/3
 329/1562 [=====>........................] - ETA: 7:49 - loss: 1.3004 - acc: 0.5334

1562/1562 [============================>.] - ETA: 0s - loss: 1.2548 - acc: 0.5488

...

782/781 [==============================] - 369s 472ms/step - loss: 0.6678 - acc: 0.7673 - val_loss: 0.7540 - val_acc: 0.7536
Epoch 15/18
264/781 [=========>....................] - ETA: 3:43 - loss: 0.6555 - acc: 0.7740

782/781 [==============================] - 361s 461ms/step - loss: 0.6564 - acc: 0.7720 - val_loss: 0.8033 - val_acc: 0.7419

Epoch 00015: saving model to cifar10_weights.15-0.803.hdf5
Epoch 16/18
207/781 [======>.......................] - ETA: 4:14 - loss: 0.6436 - acc: 0.7781

782/781 [==============================] - 365s 467ms/step - loss: 0.6392 - acc: 0.7790 - val_loss: 0.8957 - val_acc: 0.7349
Epoch 17/18
244/781 [========>.....................] - ETA: 4:01 - loss: 0.6210 - acc: 0.7832

782/781 [==============================] - 370s 473ms/step - loss: 0.6231 - acc: 0.7824 - val_loss: 0.7213 - val_acc: 0.7670
Epoch 18/18
258/781 [========>.....................] - ETA: 3:53 - loss: 0.5995 - acc: 0.7905

782/781 [==============================] - 366s 468ms/step - loss: 0.6060 - acc: 0.7893 - val_loss: 0.7434 - val_acc: 0.7651


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

from google.colab import files

files.download('DNST_model.h5')

Saved model to disk


In [0]:
model.layers[2].get_weights()[0][0][0][0]

array([-0.08926045, -0.03382662,  0.1389838 ,  0.04840288,  0.15290497,
        0.20032234, -0.01584686,  0.01852103, -0.07243107,  0.00584622,
        0.08848107, -0.18683584, -0.14112152,  0.1708244 , -0.05623038,
       -0.08167133, -0.1807296 , -0.05519202, -0.01146097,  0.12076344,
        0.34433368, -0.0590573 , -0.07619301,  0.19274437, -0.02804155,
        0.16227992, -0.07030777,  0.04280185, -0.17523837,  0.16714962,
        0.12454204,  0.1990029 ], dtype=float32)

# Step 2:- Train model on original Image


---



In [0]:
#Everthing same except Dense layer name
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3] 
dropout_rate = 0.1

input = Input(shape=(img_height, img_width, channel,),name='input_layer')
input_norm = BatchNormalization(axis=-1,name='Channelwise_norm') (input)
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same',name='Conv2D_1st')(input_norm)

block=1
First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

block=2
Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

block=3
Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

block=4
Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = my_output_layer(Last_Block)       #my_output_layer:- dense layer with different name

  
my_model = Model(inputs=[input], outputs=[output])

### Load Pre-Train Weights


In [0]:
my_model.load_weights('DNST_model.h5',by_name=True)  # Load weights for all layers except Dense layer 

In [0]:
my_model.layers[2].get_weights()[0][0][0][0]   #Weights same as Pre-trained model


array([-0.08926045, -0.03382662,  0.1389838 ,  0.04840288,  0.15290497,
        0.20032234, -0.01584686,  0.01852103, -0.07243107,  0.00584622,
        0.08848107, -0.18683584, -0.14112152,  0.1708244 , -0.05623038,
       -0.08167133, -0.1807296 , -0.05519202, -0.01146097,  0.12076344,
        0.34433368, -0.0590573 , -0.07619301,  0.19274437, -0.02804155,
        0.16227992, -0.07030777,  0.04280185, -0.17523837,  0.16714962,
        0.12454204,  0.1990029 ], dtype=float32)

In [0]:
# determine Loss function and Optimizer
my_weight_checksaver = keras.callbacks.ModelCheckpoint('my_cifar10_weights.{epoch:02d}-{val_loss:.3f}.hdf5',
                                                  monitor='val_loss', verbose=1, save_weights_only=True, period=3, mode = 'auto')

my_model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
#Total 32 epochs 

epochs=32
batch_size=64
my_history1=my_model.fit_generator(imgen.flow(x_train,y_train,batch_size=batch_size),
                      steps_per_epoch=len(x_train)/batch_size,
                      epochs=epochs, 
                      verbose=1,
                      callbacks=[my_weight_checksaver],
                      validation_data=(x_test,y_test)
                     )
#For Epoch 22/32, val_acc: 0.8863[total 40 = 18+22 Epochs ]

Epoch 1/32
782/781 [==============================] - 598s 765ms/step - loss: 0.6335 - acc: 0.7846 - val_loss: 0.7378 - val_acc: 0.7715
Epoch 2/32
122/781 [===>..........................] - ETA: 7:41 - loss: 0.5121 - acc: 0.8291

782/781 [==============================] - 581s 743ms/step - loss: 0.5034 - acc: 0.8287 - val_loss: 0.8411 - val_acc: 0.7647
Epoch 3/32
213/781 [=======>......................] - ETA: 6:38 - loss: 0.4638 - acc: 0.8416

782/781 [==============================] - 582s 744ms/step - loss: 0.4538 - acc: 0.8446 - val_loss: 0.9955 - val_acc: 0.7312

Epoch 00003: saving model to my_cifar10_weights.03-0.995.hdf5
Epoch 4/32
191/781 [======>.......................] - ETA: 6:54 - loss: 0.4268 - acc: 0.8524

782/781 [==============================] - 583s 745ms/step - loss: 0.4280 - acc: 0.8520 - val_loss: 0.6161 - val_acc: 0.8156
Epoch 5/32
239/781 [========>.....................] - ETA: 6:21 - loss: 0.4093 - acc: 0.8620

782/781 [==============================] - 583s 746ms/step - loss: 0.4049 - acc: 0.8625 - val_loss: 0.5290 - val_acc: 0.8399
Epoch 6/32
257/781 [========>.....................] - ETA: 6:07 - loss: 0.3754 - acc: 0.8713

...

782/781 [==============================] - 581s 743ms/step - loss: 0.2360 - acc: 0.9176 - val_loss: 0.5857 - val_acc: 0.8501

Epoch 00021: saving model to my_cifar10_weights.21-0.586.hdf5
Epoch 22/32
203/781 [======>.......................] - ETA: 6:45 - loss: 0.2177 - acc: 0.9246

782/781 [==============================] - 581s 742ms/step - loss: 0.2317 - acc: 0.9189 - val_loss: 0.4239 - val_acc: 0.8863
Epoch 23/32
242/781 [========>.....................] - ETA: 6:17 - loss: 0.2090 - acc: 0.9258

782/781 [==============================] - 581s 743ms/step - loss: 0.2244 - acc: 0.9211 - val_loss: 0.4861 - val_acc: 0.8640
Epoch 24/32
257/781 [========>.....................] - ETA: 6:07 - loss: 0.2065 - acc: 0.9286

782/781 [==============================] - 581s 743ms/step - loss: 0.2184 - acc: 0.9244 - val_loss: 0.4769 - val_acc: 0.8669

Epoch 00024: saving model to my_cifar10_weights.24-0.477.hdf5
Epoch 25/32
203/781 [======>.......................] - ETA: 6:45 - loss: 0.2052 - acc: 0.9280

782/781 [==============================] - 580s 742ms/step - loss: 0.2127 - acc: 0.9261 - val_loss: 0.4465 - val_acc: 0.8820
Epoch 26/32
 98/781 [==>...........................] - ETA: 7:57 - loss: 0.1947 - acc: 0.9306

In [0]:
my_model.load_weights('my_cifar10_weights.24-0.477.hdf5')
score = my_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 39s 4ms/step
Test loss: 0.4768641710996628
Test accuracy: 0.8669


In [0]:
# Continue from 25th epoch
#Initial_epoch=24
#Total Epoch= 32-24 = 8
epochs=32
batch_size=64
my_history1=my_model.fit_generator(imgen.flow(x_train,y_train,batch_size=batch_size),
                      steps_per_epoch=len(x_train)/batch_size,
                      epochs=epochs, 
                      verbose=1,
                      initial_epoch=24,
                      shuffle=True,
                      callbacks=[my_weight_checksaver],
                      validation_data=(x_test,y_test)
                     )

#For Epoch 32/32, val_acc: 0.8883 [For 50= 18+32 epochs ]

Epoch 25/32
782/781 [==============================] - 585s 749ms/step - loss: 0.2089 - acc: 0.9264 - val_loss: 0.4713 - val_acc: 0.8724
Epoch 26/32
118/781 [===>..........................] - ETA: 7:49 - loss: 0.2029 - acc: 0.9292

782/781 [==============================] - 585s 748ms/step - loss: 0.2135 - acc: 0.9259 - val_loss: 0.4743 - val_acc: 0.8731

Epoch 00026: saving model to my_cifar10_weights.26-0.474.hdf5
Epoch 27/32
162/781 [=====>........................] - ETA: 7:17 - loss: 0.2009 - acc: 0.9280

782/781 [==============================] - 585s 748ms/step - loss: 0.2053 - acc: 0.9266 - val_loss: 0.5433 - val_acc: 0.8736
Epoch 28/32
227/781 [=======>......................] - ETA: 6:31 - loss: 0.1928 - acc: 0.9336

782/781 [==============================] - 585s 748ms/step - loss: 0.2068 - acc: 0.9279 - val_loss: 0.4213 - val_acc: 0.8823
Epoch 29/32
251/781 [========>.....................] - ETA: 6:12 - loss: 0.1789 - acc: 0.9371

782/781 [==============================] - 583s 745ms/step - loss: 0.1939 - acc: 0.9321 - val_loss: 0.4675 - val_acc: 0.8837

Epoch 00029: saving model to my_cifar10_weights.29-0.468.hdf5
Epoch 30/32
201/781 [======>.......................] - ETA: 6:48 - loss: 0.1835 - acc: 0.9363

782/781 [==============================] - 583s 745ms/step - loss: 0.1913 - acc: 0.9324 - val_loss: 0.7133 - val_acc: 0.8361
Epoch 31/32
242/781 [========>.....................] - ETA: 6:19 - loss: 0.1830 - acc: 0.9366

782/781 [==============================] - 583s 745ms/step - loss: 0.1904 - acc: 0.9336 - val_loss: 0.4778 - val_acc: 0.8751
Epoch 32/32
257/781 [========>.....................] - ETA: 6:08 - loss: 0.1796 - acc: 0.9359

782/781 [==============================] - 583s 746ms/step - loss: 0.1824 - acc: 0.9364 - val_loss: 0.4271 - val_acc: 0.8883

Epoch 00032: saving model to my_cifar10_weights.32-0.427.hdf5


In [0]:
#Test the model
score = my_model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 39s 4ms/step
Test loss: 0.42713764951527117
Test accuracy: 0.8883


In [0]:
# Save the trained weights in to .h5 format
my_model.save_weights("DNST_my_model.h5")
print("Saved model to disk")

from google.colab import files
files.download('DNST_my_model.h5')
               
my_model.save('my_model.h5')
files.download('my_model.h5')

the_weights = my_model.get_weights()
np.save("my_weights_array", the_weights)
files.download('my_weights_array.npy')

Saved model to disk
